In [50]:
import pandas as pd

In [51]:
data = pd.read_csv('../GSAF5.csv', engine="python")
print(data.shape)

data.columns


(5992, 24)


Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

DATA CLEANING
Hypothesis: Male surfers in Florida have had the highest incidence of both shark attacks and deaths, with increasing frequency since the 1900 up until today

In [52]:
# Renaming columns where unintented spaces might cause problems
data.rename(columns={'Sex ': 'Sex', 'Species ': 'Species'}, inplace=True)
data.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [53]:
# Checking the types of data
data.dtypes

Case Number               object
Date                      object
Year                       int64
Type                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Name                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Time                      object
Species                   object
Investigator or Source    object
pdf                       object
href formula              object
href                      object
Case Number.1             object
Case Number.2             object
original order             int64
Unnamed: 22               object
Unnamed: 23               object
dtype: object

In [54]:
#CHECKING IF THERE ARE DUPLICATED COLUMNS, AS IT SEEMS SO FROM A FIRST LOOK TO THE DATABASE
def getDuplicateColumns(df):  
    duplicateColumnNames = []
    for x in range(df.shape[1]):
        col = df.iloc[:, x]
        for y in range(x + 1, df.shape[1]):
            otherCol = df.iloc[:, y]
            if col.equals(otherCol):
                duplicateColumnNames.append(df.columns.values[y])
            else:
                pass
 
    return duplicateColumnNames

getDuplicateColumns(data)

[]

DELETING DUPLICATED COLUMNS

In [55]:
#Case Number repeated, keeping only 1 column, checking that other two were dropped
data.loc[:, ['Case Number','Case Number.1','Case Number.2']]
print('Before: ', data.shape)
data = data.drop(['Case Number.1', 'Case Number.2'], axis=1)
print('After: ', data.shape)

Before:  (5992, 24)
After:  (5992, 22)


In [56]:
# Href and pdf repeated, keeping only 1 column, checking that other two were dropped
data.loc[:, ['pdf','href','href formula']]
print('Before: ', data.shape)
data = data.drop(['pdf', 'href formula'], axis=1)
print('After: ', data.shape)









Before:  (5992, 22)
After:  (5992, 20)


In [57]:
# Deleting Useless Columns from a qualitative analysis: These are 
# Name: Analysing data overall, not about individuals
# Date: The weather seasons are in different months in different countries, september in USA is not the same as in Australia, difficult to make comparisons so dropped
# Investigator or Source and href: Not important for investigating the hypothesis
# Injury and Type: We only care about the attack, not the details of how it affected each individual
# Original Order: Number has no value for the analysis
print('Before: ', data.shape)
data = data.drop(['Name', 'Investigator or Source', 'Date', 'Injury', 'original order', 'href', 'Type'], axis=1)
print('After: ', data.shape)




Before:  (5992, 20)
After:  (5992, 13)


In [58]:
# checking for NULL values in columns 

data.isnull().sum()

Case Number       0
Year              0
Country          43
Area            402
Location        496
Activity        527
Sex             567
Age            2681
Fatal (Y/N)      19
Time           3213
Species        2934
Unnamed: 22    5991
Unnamed: 23    5990
dtype: int64

In [59]:
# Dropping columns with more than 40% Nulls: Age, Time, Species, Unnamed:22 and Unnamed 23
cutoff_null = .6 * len(data)
data.dropna(thresh = cutoff_null, axis = 1, inplace = True)
print('After: ', data.shape)



After:  (5992, 8)


In [60]:
# Dropping row if year below 1900, only interested in the past 2 centuries
data = data[data.Year >1900]
data.shape

(5311, 8)

In [61]:
# Checking for duplicated rows, dropping them (No apparent duplicates found)
before = len(data)
data = data.drop_duplicates()
after = len(data)
print('Number of duplicate records dropped: ', str(before - after))

Number of duplicate records dropped:  0


In [63]:
#Look at data types to see what numeric values are not appearing as numeric
data.head(15)

,Case Number,Year,Country,Area,Location,Activity,Sex,Fatal (Y/N)
0,2016.09.18.c,2016,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,N
1,2016.09.18.b,2016,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,N
2,2016.09.18.a,2016,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,N
3,2016.09.17,2016,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,M,N
4,2016.09.15,2016,AUSTRALIA,Victoria,Bells Beach,Surfing,M,N
5,2016.09.15.R,2016,AUSTRALIA,Western Australia,Bunbury,Fishing,NaN,N
6,2016.09.11,2016,USA,Florida,"Ponte Vedra, St. Johns County",Wading,M,N
7,2016.09.07,2016,USA,Hawaii,"Makaha, Oahu",Swimming,F,N
8,2016.09.06,2016,NEW CALEDONIA,North Province,Koumac,Kite surfing,M,Y
9,2016.09.05.b,2016,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,F,N


In [64]:
# Filling the remaining missing values as Unknown:
data[['Country', 'Area', 'Location', 'Activity', 'Sex', 'Fatal (Y/N)']] = data[['Country', 'Area', 'Location', 'Activity', 'Sex', 'Fatal (Y/N)']].fillna('Unknown')
data.isnull().sum()

Case Number    0
Year           0
Country        0
Area           0
Location       0
Activity       0
Sex            0
Fatal (Y/N)    0
dtype: int64

In [66]:
data['Country'].value_counts()

USA                           1986
AUSTRALIA                     1121
SOUTH AFRICA                   532
PAPUA NEW GUINEA               129
BRAZIL                         100
NEW ZEALAND                     94
BAHAMAS                         90
MEXICO                          73
ITALY                           61
PHILIPPINES                     55
REUNION                         55
FIJI                            50
NEW CALEDONIA                   49
MOZAMBIQUE                      41
EGYPT                           30
JAPAN                           29
SPAIN                           29
CUBA                            28
PANAMA                          27
CROATIA                         26
IRAN                            25
SOLOMON ISLANDS                 25
HONG KONG                       24
Unknown                         22
FRENCH POLYNESIA                19
JAMAICA                         19
INDONESIA                       17
GREECE                          14
BERMUDA             

In [75]:
# Cleaning Country Column from characters etc
import re

data['Country'] = data['Country'].str.replace(' PHILIPPINES', 'PHILIPPINES')
data['Country'] = data['Country'].str.replace('Sierra Leone', 'SIERRA LEONE')
data[['Country','Area']]



,Country,Area
0,USA,Florida
1,USA,Florida
2,USA,Florida
3,AUSTRALIA,Victoria
4,AUSTRALIA,Victoria
5,AUSTRALIA,Western Australia
6,USA,Florida
7,USA,Hawaii
8,NEW CALEDONIA,North Province
9,USA,South Carolina


In [ ]:
#Cambiar Column Order

In [ ]:
#Preffer to analyse area than country as country too big, list major areas and countries